# Milvus db set up

In [1]:
from pymilvus import DataType, MilvusClient, connections, Collection
import pandas as pd

In [2]:
posts_df = pd.read_parquet('posts.parquet.gzip')

In [3]:
posts_data = posts_df[['id', 'bert_descr_vector', 'tfidf_descr_vector', 'image_vector', 'tags_vector']]
posts_data = posts_data.to_dict(orient='records')

for post in posts_data:
    for vector_name in ['bert_descr_vector', 'tfidf_descr_vector', 'image_vector', 'tags_vector']:
        post[vector_name] = post[vector_name].tolist()

In [2]:
connections.connect(
  alias="default", 
  host='localhost',
  port='19530'
)

In [2]:
m_client = MilvusClient("http://localhost:19530")

In [6]:
if m_client.has_collection('post'):
    m_client.drop_collection('post')

if m_client.has_collection('user'):
    m_client.drop_collection('user')


post_schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=False,
)

user_schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=False,
)

post_schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)

user_schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
user_schema.add_field(field_name="weight_count", datatype=DataType.INT64)

for vector_name in ['bert_descr_vector', 'tfidf_descr_vector', 'image_vector', 'tags_vector']:
    post_schema.add_field(field_name=vector_name, datatype=DataType.FLOAT_VECTOR, dim=len(posts_data[0][vector_name]))
    user_schema.add_field(field_name=vector_name, datatype=DataType.FLOAT_VECTOR, dim=len(posts_data[0][vector_name]))
    


user_index_params = m_client.prepare_index_params()
post_index_params = m_client.prepare_index_params()

for index_params in [user_index_params, post_index_params]:
    
    for vector_name in ['bert_descr_vector', 'tfidf_descr_vector', 'image_vector', 'tags_vector']:
        
        index_params.add_index(
            field_name="id",
            index_type="STL_SORT"
        )
        
        index_params.add_index(
            field_name=vector_name, 
            index_type="IVF_FLAT",
            metric_type="COSINE",
            params={ "nlist": 128 }
        )


m_client.create_collection(
    collection_name="post",
    schema=post_schema,
    index_params=post_index_params
)


m_client.create_collection(
    collection_name="user",
    schema=user_schema,
    index_params=user_index_params
)


In [7]:
m_client.get_load_state(
    collection_name="post"
)

{'state': <LoadState: Loaded>}

In [8]:
m_client.get_load_state(
    collection_name="user"
)

{'state': <LoadState: Loaded>}

In [9]:
m_client.upsert(collection_name='post', data = posts_data)

{'upsert_count': 2825, 'cost': 0}

In [3]:
res = m_client.get(collection_name='user', ids=[107])

In [4]:
res

data: ["{'tfidf_descr_vector': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0